## Parameters

In [ ]:
# ------- PARAMETERS ------- # 

RUNS = [137,138,139,140,141,142,143,147,148,149,150,151,152,153,156,157,158,159,160,161,162,163,164,167,168,169,170]

root = '/pnfs/pic.es/data/cta/LST/LST2/Data/ZFITS/'

# -------------------------- #

# other parameters
output_path   = 'file_output/'
limitDistance = 0.21 # radius we get for the neighbor pixels

## Import and functions

In [ ]:
import numpy  as np
import pandas as pd
import os

from ctapipe.instrument    import CameraGeometry
from ctapipe.visualization import CameraDisplay
from ctapipe.coordinates   import EngineeringCameraFrame
from traitlets.config      import Config
from ctapipe.io            import EventSource

from IPython.display import clear_output

import auxiliar as aux

# extracting the number of LST
LST_camera = aux.find_LST_num(root) 

# getting geometry of the camera
date, subruns = aux.search(root,RUNS[0])    
config  = Config({'LSTEventSource': {'default_trigger_type': 'ucts','allowed_tels': [1],
                                    'min_flatfield_adc': 3000,'min_flatfield_pixel_fraction': 0.8,},})  

path    = root + date+'/'+'LST-'+LST_camera+'.1.Run'+str(RUNS[0]).zfill(5)+'.'+str(0).zfill(4)+'.fits.fz'
source  = EventSource(input_url=path, config=config)
camgeom = source.subarray.tel[1].camera.geometry

# camera geometry for CaCo
camgeom2 = CameraGeometry.from_name("LSTCam")

# creating the folder if dont exist
aux.create_folder(output_path) 

Finding the neighbor pixels

In [ ]:
# first and second neighbors to pixels
print('Calculating neighbors...')
PX_N, PX_NN = aux.neighborPixels(camgeom, limitDistance)

# pixels we are interested in each run
print('Calculating neighbors we are interested in each run...')
pixelsRun = aux.neighborCluster(camgeom, camgeom2, RUNS)


In [ ]:
# create (or erase) files
for px in range(1855):
    open(output_path + 'Pixel' + str(px).zfill(4) + '.txt', 'w+')

# a list with the near and next-near neighbors
for px in range(1855):
    f = open(output_path + 'Pixel' + str(px).zfill(4) + '.txt', 'a+')
    f.write('N\n')
    f.write(str(PX_N[px]).replace(' ', '')  + '\n')
    f.write('NN\n')
    f.write(str(PX_NN[px]).replace(' ', '') + '\n')
    f.close()

## All subruns

In [ ]:
# over all runs
for run in RUNS:

    # finding the run in the folder root
    date, subruns = aux.search(root,run)    

    # to open a second run we need to use the first subrun of the run and
    # extract the timestamp of the first event

    # first subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,
            },
        })  

    path   = root + date + '/'
    path   = path + 'LST-' + LST_camera + '.1.Run' + str(run).zfill(5) + '.' + str(0).zfill(4) + '.fits.fz'
    source = EventSource(input_url=path, config=config, max_events=1)

    for i, ev in enumerate(source): 
        timestamp = int(ev.trigger.time.value)

    # following subrun configuration
    config = Config(
        {
            'LSTEventSource': {
                'default_trigger_type': 'ucts',
                'allowed_tels': [1],
                'min_flatfield_adc': 3000,
                'min_flatfield_pixel_fraction': 0.8,

          "EventTimeCalculator": {
            "dragon_module_id":133 ,
            "dragon_reference_counter": 0,
            "dragon_reference_time": timestamp,
          },
            },
        })
    
    # over all subruns
    for subrun in subruns:
        
        path = root+date+'/'+'LST-'+LST_camera+'.1.Run'+str(run).zfill(5)+'.'+str(subrun).zfill(4)+'.fits.fz'
        source = EventSource(input_url=path, config=config)

        # iterating over all events
        for i, ev in enumerate(source): 

            if i % 3000 == 0 or i == 0:
                print('Run ' + str(run) + ' Subrun ' + str(subrun) +' - '+ str(round(100 * i/53000,2)) + '%')

            pixel = None
            # we look inly to pixels in the area we are interested for the run
            for px in pixelsRun[RUNS.index(run)]:
                
                # detecting a flash in one pixel
                if max(ev.r0.tel[1].waveform[0][px][5:]) > 1000:
                    pixel = px
                    break

            if pixel != None:
                
                # writting the information of the other pixels in the file 
                # of the pixel that recieved the flash
                
                tot_px = [pixel] + PX_NN[pixel]
                f      = open(output_path + 'Pixel' + str(pixel).zfill(4) + '.txt', 'a+')

                for px in tot_px:
                    if px == tot_px[0]:
                        
                        # saving the maximum amplitude of the waveform 
                        f.write(str(max(ev.r0.tel[1].waveform[0][px][5:])))
                        
                    else:
                        f.write(',' + str(max(ev.r0.tel[1].waveform[0][px][5:])))
                f.write('\n')
                f.close()

        clear_output()
        print('Run ' + str(run) + ' Subrun ' + str(subrun) + ' - 100 % \n\n')